In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import warnings
from config import API_KEY
warnings.filterwarnings("ignore")

In [80]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='c'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'c':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'p':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return option_price

def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)


def implied_volatility(S, K, T, r, market_price, option_type='c', tol=1e-8, max_iterations=100):
    sigma = 0.5  # Initial guess
    try:
        for i in range(max_iterations):
            option_price = black_scholes(S, K, T, r, sigma, option_type)
            v = vega(S, K, T, r, sigma)
            
            price_diff = option_price - market_price
            
            if abs(price_diff) < tol:
                return sigma
            
            sigma = sigma - price_diff / v
    except:
        return 0
    # raise ValueError("Implied volatility did not converge")

S = 34.49
K = 35.00
T = 120/365
r = 0.05
market_price = 3.35

implied_vol = implied_volatility(S, K, T, r, market_price, option_type='c')
implied_vol

np.float64(0.42216145593045773)

In [88]:
def get_stock_price(symbol='AAPL', start='2023-01-09', end='2023-01-20'):
    '''
    symbol, start_da, end_da
    '''
    uri = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start}/{end}?apiKey={API_KEY}"
    res = requests.get(uri)
    data = res.json()
    df = pd.DataFrame(data['results'])
    df['t'] = df['t'].apply(lambda x: datetime.fromtimestamp(x/1000))
    df.set_index('t', inplace=True)
    return df
def get_option_by_ticker(symbol='TSLA', expiration='230113', type_='c', strike='150', start='2023-01-01', end='2023-01-11'):
    ticker = f"{symbol}{expiration}C000{strike}00"
    uri = f"https://api.polygon.io/v2/aggs/ticker/O:{ticker}/range/1/day/{start}/{end}?apiKey={API_KEY}"
    try:
        res = requests.get(uri)
        df = pd.DataFrame(res.json()['results'])
        df['t'] = df['t'].apply(lambda x: datetime.fromtimestamp(x/1000))
        df['strike'] = strike
        df['T'] = (pd.to_datetime(f"20{expiration}") - df['t']).apply(lambda x: x.days) / 365
        df.set_index('t', inplace=True)
        df = df.apply(lambda x: x.astype(float), axis=1)
        df['ticker'] = ticker
        df['symbol'] = symbol
        df['type'] = type_
        return df
    except KeyError as e:
        return res.json()


In [139]:
get_option_by_ticker()

,v,vw,o,c,h,l,n,strike,T,ticker,symbol,type
t,,,,,,,,,,,,
2023-01-04 13:00:00,92.0,97.6463,94.58,97.28,99.45,94.58,11.0,150.0,0.021918,TSLA230113C00015000,TSLA,c
2023-01-06 13:00:00,20.0,94.0300,93.96,94.10,94.10,93.96,2.0,150.0,0.016438,TSLA230113C00015000,TSLA,c
2023-01-09 13:00:00,24.0,105.1763,103.30,104.40,107.40,103.30,6.0,150.0,0.008219,TSLA230113C00015000,TSLA,c
2023-01-11 13:00:00,4.0,107.2375,106.20,109.00,109.00,106.20,4.0,150.0,0.002740,TSLA230113C00015000,TSLA,c


In [138]:
# i.keys() for i in res.json()['results']: [dict_keys(['day', 'details', 'greeks', 'implied_volatility', 'open_interest', 'underlying_asset'])
request = "https://api.polygon.io/v3/snapshot/options/TSLA?strike_price=150&limit=100&apiKey={API_KEY}"
res = requests.get(request).json()
implied_vol = [i['implied_volatility'] for i in res['results']]
contract_type = [i['details']['contract_type'] for i in res['results']]
expiration_date = [i['details']['expiration_date'] for i in res['results']]
strike_price = [i['details']['strike_price'] for i in res['results']]
delta = [i['greeks']['delta'] for i in res['results']]
gamma = [i['greeks']['gamma'] for i in res['results']]
theta = [i['greeks']['theta'] for i in res['results']]
vega = [i['greeks']['vega'] for i in res['results']]
open_interest = [i['open_interest'] for i in res['results']]
last_updated = [datetime.fromtimestamp(i['day']['last_updated']/1000000000) for i in res['results'][:-2]]
close = [i['day']['close'] for i in res['results'][:-2]]
open = [i['day']['open'] for i in res['results'][:-2]]
high = [i['day']['high'] for i in res['results'][:-2]]
low = [i['day']['low'] for i in res['results'][:-2]]
prev_close = [i['day']['previous_close'] for i in res['results'][:-2]]

df = pd.DataFrame()
df['implied_volatility'] = implied_vol
df['contract_type'] = contract_type
df['expiration_date'] = expiration_date
df['strike_price'] = strike_price
df['delta'] = delta
df['gamma'] = gamma
df['theta'] = theta
df['vega'] = vega
df['open_interest'] = open_interest
df['last_updated'] = last_updated + [0, 0]
df['close'] = close + [0, 0]
df['open'] = open + [0, 0]
df['high'] = high + [0, 0]
df['low'] = low + [0, 0]
df['prev_close'] = prev_close + [0, 0]
df.head(1)

,implied_volatility,contract_type,expiration_date,strike_price,delta,gamma,theta,vega,open_interest,last_updated,close,open,high,low,prev_close
0,4.84897,call,2024-07-26,150,0.904065,0.002589,-3.840698,0.020092,130,2024-07-24 12:00:00,66.55,75.0,75.0,66.31,97.45


In [108]:
soup['results']

[{'day': {'change': -0.07,
   'change_percent': -13.5,
   'close': 0.45,
   'high': 0.7,
   'last_updated': 1721793600000000000,
   'low': 0.43,
   'open': 0.7,
   'previous_close': 0.52,
   'volume': 669,
   'vwap': 0.5181},
  'details': {'contract_type': 'call',
   'exercise_style': 'american',
   'expiration_date': '2024-08-16',
   'shares_per_contract': 100,
   'strike_price': 10,
   'ticker': 'O:EVRI240816C00010000'},
  'greeks': {'delta': 0.37870924414516116,
   'gamma': 0.17931917867632616,
   'theta': -0.01883115387346594,
   'vega': 0.008136419701186903},
  'implied_volatility': 0.9612814484408605,
  'open_interest': 6795,
  'underlying_asset': {'ticker': 'EVRI'}},
 {'day': {'change': 0,
   'change_percent': 0,
   'close': 1.95,
   'high': 1.95,
   'last_updated': 1721678400000000000,
   'low': 1.95,
   'open': 1.95,
   'previous_close': 1.95,
   'volume': 10,
   'vwap': 1.95},
  'details': {'contract_type': 'put',
   'exercise_style': 'american',
   'expiration_date': '2024-0